In [1]:
import sys
sys.path.append('../')

In [2]:
from application.review_obj_class import ReviewObj
from application.namespan_class import NameSpanGenerator
import pandas as pd
import os
%pprint

Pretty printing has been turned OFF


In [3]:
directory = "../../aps_reviews_1000/"
filenames = os.listdir(directory)
reviews = ((file.split('.')[0], open(directory + file).read()) for file in filenames)

In [4]:
review_list = [ReviewObj(file, txt) for (file, txt) in reviews]

In [5]:
spans = NameSpanGenerator.generate(review_list[0])

In [6]:
review_list[0].review_id

'125720170'

In [7]:
spans[0].name

'Co-operativo PFblishing Company'

In [10]:
# for e, x in enumerate(review_list):
#     try:
#         NameSpanGenerator.generate(x)
#     except:
#         print(e, x.review_id)
#         raise
#         print()

In [11]:
spans = [NameSpanGenerator.generate(x) for x in review_list]

In [12]:
len(spans)

1003

In [20]:
ex_review.cleaned_toks

['Mr.', 'Kenneth', 'Grahame', "'s", 'classic', 'of', ',', '``', '1', 'The', 'Golden', 'Age', ',', '``', 'appeared', 'some', 'three', 'or', 'four', 'years', 'ago', 'in', 'a', 'holiday', 'edition', ',', 'with', 'illustrations', 'by', 'Mr.', 'Maxfield', 'Parrish', '.', 'A', 'happier', 'combination', 'of', 'author', 'and', 'illustrator', 'could', 'hardly', 'have', 'been', 'hit', 'upon', ',', 'but', 'it', 'was', 'quite', 'evident', 'that', 'the', 'half-tone', 'process', 'employed', 'in', 'repro', '.', 'ducing', 'the', 'pictures', 'did', 'scant', 'justice', 'to', 'the', 'beauty', 'of', 'Mr.', 'Parrish', "'s", 'originals', '.', 'We', 'are', 'glad', 'to', 'note', 'that', 'the', 'publisher', '(', 'John', 'Lane', ')', 'has', 'now', 'brought', 'out', 'a', 'new', 'edition', 'of', 'the', 'book', ',', 'with', 'the', 'illustrations', 'reproduced', 'in', 'photogravure', '.', 'The', 'result', 'is', 'a', 'great', 'improvement', 'in', 'every', 'way', 'over', 'the', 'original', 'edition', ',', 'the', 'vol

In [28]:
# this code is stolen from Ben Naismith, Ling PhD student, who in turn adapted it from Dr. Na-Rae Han

# Finding word in a tokenized text
def concord_toks(text, word, num):
    padding = [('')] * num             # non-word padding, in the specified size
    padded = padding + text + padding   # pad in front and back
    outlist = []
    for i in range(len(padded)):
        if padded[i] == word:
            outlist.append((padded[i-num:i], padded[i], padded[i+1:i+1+num]))
    return outlist

def flatten(outlist):
    flatlist = []
    for x in outlist:
        flatlist.append((' '.join(x[0]), x[1], ' '.join(x[2])))
    return flatlist

def concordance(text, word, num):
    return flatten(concord_toks(text, word, num))

In [29]:
concordance(ex_review.cleaned_toks, 'Grahame', 3)

[(' Mr. Kenneth', 'Grahame', "'s classic of")]

In [35]:
ex_people[0].name_parts

['mr.', 'kenneth', 'grahame']

In [36]:
ex_people[0].name_part_count

3

In [38]:
ex_people[0].full_name

'Mr. Kenneth Grahame'

In [47]:
toks = []
for person in ex_people:
    num_toks = person.name_part_count
    for e, x in enumerate(ex_review.cleaned_toks):
        if x.lower() == person.name_parts[0]:
            if person.full_name.split() == ex_review.cleaned_toks[e:e + num_toks]:
                toks.append((e, e+num_toks))

In [48]:
toks

[(0, 3), (29, 32), (70, 72), (185, 187), (196, 199), (204, 207), (212, 216), (226, 229)]

In [50]:
def get_toks_for_people(ReviewObj):
    for person in ReviewObj.person_names:
        num_toks = person.name_part_count
        for e, x in enumerate(ReviewObj.cleaned_toks):
            if x.lower() == person.name_parts[0]:
                if person.full_name.split() == ReviewObj.cleaned_toks[e:e + num_toks]:
                    person.review_loc_toks = (e, e+num_toks)
    return ReviewObj

In [52]:
ex_review_2 = get_toks_for_people(ex_review)

In [55]:
for x in ex_review_2.person_names:
    print(x.review_loc_toks)
    print(ex_review_2.cleaned_toks[x.review_loc_toks[0]:x.review_loc_toks[1]])

(0, 3)
['Mr.', 'Kenneth', 'Grahame']
(29, 32)
['Mr.', 'Maxfield', 'Parrish']
(70, 72)
['Mr.', 'Parrish']
(185, 187)
['Mr.', 'Swinburne']
(196, 199)
['Mr.', 'Arthur', 'Symons']
(204, 207)
['Mr.', 'Havelock', 'Ellis']
(212, 216)
['Mr.', 'A.', 'Wilson', 'Verity']
(226, 229)
['Mr.', 'George', 'Saintsbury']


In [56]:
for e, x in enumerate(review_list):
    review_list[e] = get_toks_for_people(x)

In [58]:
print(review_list[0].cleaned_text)

LITERARY NOTES.  - The Chautauqua department of " Wide Awake " is now published separately.  - Mr. Charles Dudley Warner has written for " The North American Review " a paper on prison .  - Mr. Vedder's illustrations of the " Rubfilyfit " have found a great admirer in the Queen of Italy.  - Cassell & Co. sill shortly publish a " Dictionary of English, " by Sidney J. Low and F. S. Pulling.  - Professor Thorold Rogers is writing a worL on the " of the Privileges of British Citizenship. "   -   " Mollie Carew " is the title of a pleasant story by A. M. W., just published by E. P. Dutton & Co., New Yor ' .   -  . - rs. (MISS i ) nas written tne article on Mrs. Browning for the new " Biograpbical Dictionary. "  - Chatto & Windus, the Engllsh publishers, will issue during the coming season the collected essays of Mr. Swinburne.  - Dr. William ' M. Taylor's " Life of John Knox " is announced for early by A. C. Armstrong & Sons, New York.  - The " Athenmum " reports that Mark Twain will vis t 

In [59]:
for x in review_list[0].person_names:
    print(x)
    print(x.review_loc_toks)
    print(review_list[0].cleaned_toks[x.review_loc_toks[0]:x.review_loc_toks[1]])

Mr. Charles Dudley Warner
(18, 22)
['Mr.', 'Charles', 'Dudley', 'Warner']
Mr. Vedder
(37, 39)
['Mr.', 'Vedder']
Professor Thorold Rogers
(82, 85)
['Professor', 'Thorold', 'Rogers']
Mrs. Browning
(143, 145)
['Mrs.', 'Browning']
Mr. Swinburne
(171, 173)
['Mr.', 'Swinburne']
Dr. William
(175, 177)
['Dr.', 'William']
Dr. Geor6 Ebers
(281, 284)
['Dr.', 'Geor6', 'Ebers']
Mr. Julian Hawthorne


AttributeError: 'PersonName' object has no attribute 'review_loc_toks'